In [3]:
BANNER = """
MMWXkolccc:::::::::::::::::::::cclokXWMM
MXd,.                              .,dXM
Xc.                            ..... .cX
k.                            .',;,.. .k
d.    ....................... .',;,.. .d
d.    'dOOOOOOOOOOOOOOOOOOOkdc'.....  .d
d.    ,0MMMMMMMMMMMMMMMMMMMMMWO;.     .d
d.    ,0MMMXdcccccccccccclOWMMWd.     .d
d.    ,0MMMO'             cXMMWx.     .d
d.    ,0MMMXocccccccccccclkWMMWd.     .d
d.    ,0MMMMMWWWWWWWWMMWMMMMMWK:.     .d
d.    ,0MMMWK000000000XWMMMNOo,.      .d
d.    ,0MMM0;.........,xNMMXo.        .d
d.    ,0MMM0,          .lKMMWO;.      .d
d.    'kXXXk'           .;kXXXO:.     .d
x.    ..'''..             .''''..     .x
0,                                    ,0
Wk,.                                .,kW
MWKd:'............................':dKWM
MMMMNKOxxddddddddddddddddddddddxxOKNMMMM
"""

In [4]:
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.3/780.3 kB 3.6 MB/s eta 0:00:00a 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [75]:
import faiss
from faiss import write_index, read_index
from sentence_transformers import SentenceTransformer
import math
import string
import pandas as pd
import requests
import random
from urllib.parse import urlencode
import os
import numpy as np
from catboost import CatBoostRanker, Pool
from functools import partial
import json
import pyarrow.parquet as pq
import time
from fastparquet import ParquetFile
import numpy.random as npr


# Загрузка данных

In [76]:
start_time = time.time()

In [77]:
def download_file(pk, local_name):
    base_url = \
        'https://cloud-api.yandex.net/v1/disk/public/resources/download?'

    final_url = base_url + urlencode(dict(public_key=pk))
    response = requests.get(final_url)
    download_url = response.json()['href']

    download_response = requests.get(download_url)
    with open(local_name, 'wb') as f:
        f.write(download_response.content)
        print(f'File {local_name} downloaded')


def download_from_yandex_disk():
    print(BANNER)

    files = {
        'features.parquet': 'https://disk.yandex.ru/d/W_qJitz4dZGzAg',
        'videos.parquet': 'https://disk.yandex.ru/d/JXz-oDfKFgm2Dw',
        'automarkup.parquet': 'https://disk.yandex.ru/d/vP0FzQHdtxsz4Q',
        'manualmarkup.csv': 'https://disk.yandex.ru/d/hDztN1rgW0JNjw'
    }

    filenames, filenames_to_delete = files.keys(), []
    for filename in filenames:
        if os.path.exists(filename):
            filenames_to_delete += [filename]
    for filename in filenames_to_delete:
        del files[filename]

    print(f'Will be download {len(files)} files')
    for filename, link in files.items():
        print(f'{filename} downloading in progress')
        download_file(link, filename)
        print(f'{filename} downloaded')

In [4]:
download_from_yandex_disk()

NameError: name 'BANNER' is not defined

In [5]:
seed = 42
np.random.seed(seed)



# Формирование базы векторов

In [78]:
candidates = pd.read_parquet(
    'videos.parquet',
    engine='fastparquet',
    columns=['video_id', 'video_title']
)

In [79]:
candidates = candidates.sample(n=7_000_000, replace=True, random_state=seed)

In [80]:
corpus = candidates['video_title'].apply(lambda x: x.lower()).values
video_ids = candidates['video_id'].values
del candidates

In [81]:
st_model = SentenceTransformer(
    'cointegrated/rubert-tiny2',
    device='cuda'
)

In [82]:
# когда прогоните один раз у вас на диске уже будет сохранен faiss индекс
# можно поставить значение True, чтобы сэкономить время на формирование индекса
use_formed_index = False

In [83]:
d = 312
if not use_formed_index:
    cpu_index = faiss.IndexFlatL2(d)
    cpu_index.is_trained, cpu_index.ntotal

In [84]:
# если уже есть файл ind2videoid для вашего faiss индекса - True
use_formed_id_mapping = False

if not use_formed_id_mapping:
    ind2videoid = {ind: video_id for ind, video_id in enumerate(video_ids)}
    with open('ind2videoid.json', 'w+') as f:
        json.dump(ind2videoid, f, indent=4)
else:
    with open('ind2videoid.json', 'r') as f:
        ind2videoid = json.load(f)

In [85]:
batch_size = 100000
num_batches = math.ceil(len(corpus) / batch_size)

In [86]:
if not use_formed_index:
    try:
        for i in range(15):
            # формируем батч
            start, end = i * batch_size, (i + 1) * batch_size
            corpus_batch = corpus[start:end]

            # считаем вектора для всех предложений в батче
            embeddings = st_model.encode(
                corpus_batch,
                batch_size=1000,
                show_progress_bar=True
            )

            # добавляем новые батч векторов в индекс и сохраняем его
            cpu_index.add(embeddings)
            write_index(cpu_index, 'candidates.index')

            print(f'batch: {i + 1} / {num_batches}, vectors: {cpu_index.ntotal}')

            # чистим ОЗУ
            del embeddings
    except KeyboardInterrupt:
        print('Остановлено пользователем')
        try:
            del embeddings
        except:
            pass

Batches:   0%|          | 0/100 [00:00<?, ?it/s]

batch: 1 / 70, vectors: 100000


Batches:   0%|          | 0/100 [00:00<?, ?it/s]

batch: 2 / 70, vectors: 200000


Batches:   0%|          | 0/100 [00:00<?, ?it/s]

...

batch: 13 / 70, vectors: 1300000


Batches:   0%|          | 0/100 [00:00<?, ?it/s]

batch: 14 / 70, vectors: 1400000


Batches:   0%|          | 0/100 [00:00<?, ?it/s]

batch: 15 / 70, vectors: 1500000


In [87]:
# чистим ОЗУ
if not use_formed_index:
    del cpu_index

# Подбор кандидатов по базе векторов

In [88]:
automarkup = pd.read_parquet(
    'automarkup.parquet',
    engine='fastparquet'
)

In [89]:
automarkup = automarkup[~automarkup['query'].isna()]
automarkup['query'] = automarkup['query'].apply(lambda x: x.lower())

In [90]:
n = 1000
top_n = automarkup['query'].value_counts()[:int(2*n)].index.to_list()
other = np.array(automarkup['query'].value_counts()[int(2*n):].index.to_list())
random_n = np.random.choice(other, size=n, replace=False).tolist()
queries = top_n + random_n
query2ind = {q: i for i, q in enumerate(queries)}

In [91]:
# когда прогоните один раз у вас на диске кандиды уже будут сохранены
# можете поставить значение True, чтобы сэкономить время
use_formed_candidates = False

In [92]:
qembeddings = st_model.encode(
    queries,
    batch_size=1000,
    show_progress_bar=True
)

search_cpu_index = read_index('candidates.index')
search_cpu_index.is_trained, search_cpu_index.ntotal

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

(True, 1500000)

In [93]:
generated_cand_name = 'generated_candidates.parquet'
if not use_formed_candidates:
    topk = 300
    distance, faiss_ind = search_cpu_index.search(qembeddings, topk)

    generated_cand = {
        'query': [],
        'video_id': []
    }

    for i, q in enumerate(queries):
        vids = faiss_ind[i]
        generated_cand['video_id'] += [ind2videoid[v] for v in vids]
        generated_cand['query'] += [q] * len(vids)

    generated_cand = pd.DataFrame(generated_cand)

    generated_cand.to_parquet(
        generated_cand_name,
        engine='fastparquet'
    )
else:
    generated_cand = pd.read_parquet(
        generated_cand_name,
        engine='fastparquet'
    )

# Формирование таргета по авторазметке

In [94]:
automarkup['target'] = [1] * automarkup.shape[0]
candidates_with_target = generated_cand.merge(
    automarkup[['query', 'video_id', 'target']],
    how='left',
    left_on=['query', 'video_id'],
    right_on=['query', 'video_id']
)
candidates_with_target['target'] = candidates_with_target['target'].fillna(0)

In [95]:
# очищаем ОЗУ
del generated_cand
del search_cpu_index
del automarkup
del st_model
del qembeddings

In [96]:
# здесь мы используем признаки актуальные на состояние 2-ого мая 2023 года
# вы можете использовать признаки наиболее актуальные для вашего кандидата

features_parquet = pq.ParquetFile('features.parquet')
features, filter_date = None, '2023-05-02'
for batch in features_parquet.iter_batches():
    tmp = batch.to_pandas()
    if features is None:
        features = tmp[tmp['report_date'] == filter_date]
        
    else:
        features = pd.concat([
            features, 
            tmp[tmp['report_date'] == filter_date]
        ], axis=0)
        print(features)

# для baseline выбросим категориальные признаки и datetime признаки
# в своем решении вы сами можете решить использовать их или нет
features = features.drop(
    [
        'v_channel_reg_datetime',
        'v_channel_type',
        'v_pub_datetime'
    ], 
    axis=1
)

             video_id  ... v_cr_click_comment_1_days
0       video_1240127  ...                       0.0
1       video_3266704  ...                       0.0
2      video_26852455  ...                       0.0
3      video_14041310  ...                       0.0
4       video_6944943  ...                       0.0
...               ...  ...                       ...
65531  video_30694590  ...                       0.0
65532   video_7475214  ...                       0.0
65533   video_9633916  ...                       0.0
65534  video_26926317  ...                       0.0
65535  video_30335355  ...                       0.0

[131072 rows x 29 columns]
             video_id  ... v_cr_click_comment_1_days
0       video_1240127  ...                       0.0
1       video_3266704  ...                       0.0
2      video_26852455  ...                       0.0
3      video_14041310  ...                       0.0
4       video_6944943  ...                       0.0
...               

In [25]:
features.describe()

,total_comments,v_year_views,v_month_views,v_week_views,v_day_views,v_likes,v_dislikes,v_duration,v_cr_click_like_7_days,v_cr_click_dislike_7_days,v_cr_click_vtop_7_days,v_cr_click_long_view_7_days,v_cr_click_comment_7_days,v_cr_click_like_30_days,v_cr_click_dislike_30_days,v_cr_click_vtop_30_days,v_cr_click_long_view_30_days,v_cr_click_comment_30_days,v_cr_click_like_1_days,v_cr_click_dislike_1_days,v_cr_click_vtop_1_days,v_cr_click_long_view_1_days,v_cr_click_comment_1_days
count,7.184210e+06,7.184210e+06,7.184210e+06,7.184210e+06,7.184210e+06,7.184210e+06,7184210.0,7.184210e+06,7.184210e+06,7184210.0,7.184210e+06,7.184210e+06,7.184210e+06,7.184210e+06,7184210.0,7.184210e+06,7.184210e+06,7.184210e+06,7.184210e+06,7184210.0,7.184210e+06,7.184210e+06,7.184210e+06
mean,2.325137e-01,1.607640e+02,2.276471e+01,5.813148e+00,7.982732e-01,1.736795e-02,0.0,8.435202e+05,3.119593e-04,0.0,7.294567e-04,6.735471e-02,1.739824e-04,7.415463e-04,0.0,1.721477e-03,1.439861e-01,4.641137e-04,9.024995e-05,0.0,1.939347e-04,2.246660e-02,5.269781e-05
std,4.354884e+01,1.339215e+04,3.102093e+03,1.065088e+03,1.713401e+02,1.683155e+00,0.0,6.045033e+06,1.422829e-02,0.0,2.188947e-02,2.260601e-01,1.174265e-02,1.948197e-02,0.0,3.034070e-02,3.054793e-01,1.856462e-02,8.386829e-03,0.0,1.210393e-02,1.375899e-01,9.299911e-03
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,1.046990e+05,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,4.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,2.527730e+05,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,1.500000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,7.761770e+05,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00
max,1.102180e+05,1.260422e+07,3.013797e+06,1.554484e+06,2.624620e+05,1.443000e+03,0.0,2.147484e+09,1.000000e+00,0.0,1.000000e+00,1.000000e+00,8.000000e+00,1.000000e+00,0.0,1.000000e+00,1.000000e+00,1.020000e+01,1.000000e+00,0.0,1.000000e+00,1.000000e+00,1.300000e+01


# Формирование датасета с признаками

In [97]:
full_df = candidates_with_target.merge(
    features,
    how='inner',
    left_on='video_id',
    right_on='video_id'
)
del features
full_df = full_df.drop('report_date', axis=1)
full_df = full_df.drop_duplicates()

In [98]:
groups_to_drop = []
full_df['group_id'] = full_df.groupby(['query']).ngroup()
for group in full_df['group_id'].unique():
    part_df = full_df[full_df['group_id'] == group]
    target_sum = part_df['target'].values.sum()
    if target_sum <= 0:
        groups_to_drop += [group]
full_df = full_df[~full_df['group_id'].isin(groups_to_drop)]

In [99]:
groups = pd.Series(full_df['group_id'].unique())
permutation = groups.sample(frac=1, random_state=seed)
train_groups, val_groups, test_groups = np.split(
    permutation,
    [int(0.75 * len(permutation)), int(0.90 * len(permutation))]
)

/home/jupyter/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


In [100]:
train_df = full_df[full_df['group_id'].isin(train_groups)]
val_df = full_df[full_df['group_id'].isin(val_groups)]
test_df = full_df[full_df['group_id'].isin(test_groups)]

In [101]:
train_df = train_df.sort_values('group_id')
val_df = val_df.sort_values('group_id')
test_df = test_df.sort_values('group_id')

In [102]:
metainfo_columns = ['query', 'video_id', 'target', 'group_id']

X_train = train_df.drop(metainfo_columns, axis=1)
y_train, g_train = train_df['target'], train_df['group_id']

X_val = val_df.drop(metainfo_columns, axis=1)
y_val, g_val = val_df['target'], val_df['group_id']

X_test = test_df.drop(metainfo_columns, axis=1)
y_test, g_test = test_df['target'], test_df['group_id']

In [ ]:
funny = []

In [107]:
train_df

,query,video_id,target,v_category,total_comments,v_year_views,v_month_views,v_week_views,v_day_views,v_likes,v_dislikes,v_duration,v_cr_click_like_7_days,v_cr_click_dislike_7_days,v_cr_click_vtop_7_days,v_cr_click_long_view_7_days,v_cr_click_comment_7_days,v_cr_click_like_30_days,v_cr_click_dislike_30_days,v_cr_click_vtop_30_days,v_cr_click_long_view_30_days,v_cr_click_comment_30_days,v_cr_click_like_1_days,v_cr_click_dislike_1_days,v_cr_click_vtop_1_days,v_cr_click_long_view_1_days,v_cr_click_comment_1_days,group_id
75348,20 сезон битва экстрасенсов,video_21332182,0.0,Телепередачи,0,1619,237,49,11,0,0,2957039,0.0,0.0,0.0,0.489796,0.0,0.0,0.0,0.0,0.481013,0.0,0.0,0.0,0.0,0.454545,0.0,39
77780,20 сезон битва экстрасенсов,video_12171477,0.0,Животные,0,0,0,0,0,0,0,69823,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,39
75892,20 сезон битва экстрасенсов,video_18099484,0.0,Телепередачи,0,0,0,0,0,0,0,2895767,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,39
270309,20 сезон битва экстрасенсов,video_13072123,0.0,Сериалы,0,1334,98,33,2,0,0,2813034,0.0,0.0,0.0,0.575758,0.0,0.0,0.0,0.0,0.561224,0.0,0.0,0.0,0.0,0.500000,0.0,39
270476,20 сезон битва экстрасенсов,video_9425646,0.0,Мультфильмы,0,3,0,0,0,0,0,1439970,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95537,экстросенсы ведут расследование,video_14768255,0.0,Юмор,0,0,0,0,0,0,0,166059,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,2965
95559,экстросенсы ведут расследование,video_28833931,0.0,Технологии и интернет,0,0,0,0,0,0,0,212767,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,2965
94211,экстросенсы ведут расследование,video_30861486,0.0,Техника и оборудование,0,0,0,0,0,0,0,151139,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,2965
316769,экстросенсы ведут расследование,video_27518340,0.0,Разное,0,0,0,0,0,0,0,1309629,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,2965


In [104]:
train = Pool(
    data=X_train.values,
    label=y_train.values,
    group_id=g_train.values,
    feature_names=X_train.columns.to_list()
)

val = Pool(
    data=X_val.values,
    label=y_val.values,
    group_id=g_val.values,
    feature_names=X_val.columns.to_list()
)

test = Pool(
    data=X_test.values,
    label=y_test.values,
    group_id=g_test.values,
    feature_names=X_test.columns.to_list()
)

CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=0]="Телепередачи": Cannot convert 'b'\xd0\xa2\xd0\xb5\xd0\xbb\xd0\xb5\xd0\xbf\xd0\xb5\xd1\x80\xd0\xb5\xd0\xb4\xd0\xb0\xd1\x87\xd0\xb8'' to float

# Обучение модели

In [34]:
task_type = 'GPU'
metric_period = 250

parameters = {
    'task_type': task_type,
    'verbose': False,
    'random_seed': seed,
    'loss_function': 'QueryRMSE',
    'learning_rate': 0.001,
    'l2_leaf_reg': 30,
    'iterations': 4000,
    'max_depth': 3,
}

In [35]:
model = CatBoostRanker(**parameters)
model = model.fit(
    train,
    eval_set=val,
    plot=True,
    use_best_model=True,
    metric_period=metric_period
)
model.save_model('ranker.ckpt')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [ ]:
train

# Измерение метрик

In [36]:
def _metrics_at(at, model, pool, metric='NDCG'):
    metric = metric + f':top={at}'
    eval_metrics = model.eval_metrics(pool, metrics=[metric])
    best_metrics = {}
    for key in eval_metrics.keys():
        best_metrics[key] = eval_metrics[key][model.best_iteration_]
        print(key)
    return best_metrics


metrics_train_at = partial(
    _metrics_at,
    model=model, 
    pool=train
)

metrics_val_at = partial(
    _metrics_at, 
    model=model, 
    pool=val
)

metrics_test_at = partial(
    _metrics_at, 
    model=model, 
    pool=test
)

In [37]:
metrics_train_at(1), metrics_train_at(5)

NDCG:top=1;type=Base
NDCG:top=5;type=Base


({'NDCG:top=1;type=Base': 0.65}, {'NDCG:top=5;type=Base': 0.7766176874395714})

In [38]:
metrics_val_at(1), metrics_val_at(5)

NDCG:top=1;type=Base
NDCG:top=5;type=Base


({'NDCG:top=1;type=Base': 0.53125},
 {'NDCG:top=5;type=Base': 0.6703277657198883})

In [39]:
metrics_test_at(1), metrics_test_at(5)

NDCG:top=1;type=Base
NDCG:top=5;type=Base


({'NDCG:top=1;type=Base': 0.7272727272727273},
 {'NDCG:top=5;type=Base': 0.8026431771683179})

In [96]:
model.predict(X_test[:1])

array([-0.02178205])

In [71]:
# очищаем ОЗУ
del model
del full_df
del train_df, val_df, test_df
del train, val, test
del X_train, y_train, g_train
del X_val, y_val, g_val
del X_test, y_test, g_test

In [73]:
end_time = time.time()
total_time = (end_time - start_time) / 60
print(f'Общее время работы baseline: {int(total_time)} минут')

Общее время работы baseline: 33 минут


NameError: name 'val' is not defined